In [ ]:
#import delle librerie
import traffic
import pandas as pd
import datetime
import os
import numpy as np
from traffic import *

In [ ]:
# dataframe dei 30 più importanti aeroporti d'europa
# impostare il path in base a dove viene salvato il file csv 
airport_most_important = pd.read_csv('/Users/mirkopapadopoli/Desktop/Stage/30aeroporti.csv', delimiter = ";")  

# Ciclo per creare la lista di date

In [ ]:
#creo il range di date utilizzate dal ciclo principale che richiama tutte le funzioni 
start_date = datetime.date(2016, 8, 3)
number_of_days = 2

date_list = []
for day in range(number_of_days):
    a_date = (start_date + datetime.timedelta(days = day)).isoformat()
    date_list.append(a_date)

print(date_list)

# Ciclo per creare le cartelle dove salvare i dati

In [ ]:
#creazione cartelle
# modificare il percorso (dir) in base a dove si vuole che vengano create le cartelle

dir = '/Users/mirkopapadopoli/Desktop/Data/'

for i in range(len(date_list)):
    if not os.path.exists(dir + date_list[i]):
        os.mkdir(dir + date_list[i])

# Estrazione Traiettorie

In [ ]:
#funzione che estrae i dati da OpenSky
def extract_trajectory(starts, airports):
    try:
        trajectory = opensky.history(
            start = starts,
            airport = airports
        ).data
        return trajectory
    except AttributeError: 
        return 'Error'
    

In [ ]:
#funzione che permette l'estrazione dei dati ottimizzata
def extract_data2(date):
    date_list = date
    list_name_airport = list(airport_most_important.airport_name)
    list_icao = list(airport_most_important.icao)

    print('Data:', date_list)
    for j in range(len(list_icao)):
        print('Aeroporto:', list_icao[j])
        x = extract_trajectory(date_list, list_icao[j])
        if not isinstance(x, str):
            list_name_airport[j] = x
        
        
    return list_name_airport

# Eliminazione Duplicati

In [ ]:
#funzione che mi permette di pulire i dati dai duplicati
def clean_data(list_name_airport):
    not_empty_airports = []
    name_airport_with_data = []
    for i in range(len(airport_most_important)):
        if not isinstance(list_name_airport[i], str):
            not_empty_airports.append(list_name_airport[i])
            name_airport_with_data.append(airport_most_important.icao[i])
    try:
        df_total = pd.concat(not_empty_airports)
        not_empty_airports.insert(len(not_empty_airports), df_total.drop_duplicates(keep=False))
    except: 
        print('Empty')
        not_empty_airports = []
        name_airport_with_data = []

    return name_airport_with_data, not_empty_airports

# Save Data

In [ ]:
#funzione che mi permette di salvare i dati 
# modificare il percorso in base a quello scelto in precedenza
def save_data(not_empty_airports, name_airport_with_data, x):
    
    if (len(not_empty_airports)==0):
        return
    
    ris_1g_list = []
    temp_airport = not_empty_airports.copy()

    for i in range(len(not_empty_airports)-1):
        ris_1g_list.insert(len(ris_1g_list), result_creator(temp_airport))
        ris_1g_list[i].to_csv('/Users/mirkopapadopoli/Desktop/Data/'+str(date_list[x])+'/'+str(name_airport_with_data[i])+'-'+str(date_list[x])+'.csv')
        temp_airport = temp_airport[1:] + [temp_airport[0]]

In [ ]:
#funzione che mi permette di switchare il df
def result_creator(list_airport):
    dataframe = pd.concat(list_airport[1:]).drop_duplicates(keep = False)
    return dataframe

# Funzione principale che richiama le sotto-funzioni

In [ ]:
#funzione che permette di far girare tutto lo script sulla base dei giorni dati in input
#x = indice per saltare alla cartella successiva
x = 0
for i in range(len(date_list)):
    list_name_airport = extract_data2(date_list[i])
    name_airport_with_data, not_empty_airports = clean_data(list_name_airport)
    save_data(not_empty_airports,name_airport_with_data,x)
    x +=1 